# eda003

In [1]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
import itertools

import pandas as pd
import numpy as np

In [2]:
@dataclass
class Cfg:
    exp_name = "eda001"
    input_dir = os.getenv('INPUT_DIR')
    output_dir = os.getenv('OUTPUT_DIR')
    prep_dir = os.getenv("PREP_DIR")
cfg = Cfg()

In [3]:
train = pd.read_csv(cfg.input_dir + "train.csv")
labels = pd.read_csv(cfg.input_dir + "train_labels.csv")
test = pd.read_csv(cfg.input_dir + "test.csv")
sub = pd.read_csv(cfg.input_dir + "sample_submission.csv")

In [9]:
train0_4 = train[train["level_group"]=="0-4"]
train5_12 = train[train["level_group"]=="5-12"]
train13_22 = train[train["level_group"]=="13-22"]

In [5]:
train.nunique()

session_id          11779
index               20348
elapsed_time      3804019
event_name             11
name                    6
level                  23
page                    7
room_coor_x       9120274
room_coor_y       7047316
screen_coor_x       35331
screen_coor_y       56858
hover_duration      19474
text                  594
fqid                  127
room_fqid              19
text_fqid             126
fullscreen              0
hq                      0
music                   0
level_group             3
dtype: int64

# 各グループの始点・終点

In [10]:
train[train["level_group"]=="0-4"].groupby(["session_id"]).head(1).head(10)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
881,20090312433251036,0,0,cutscene_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
2712,20090314121766812,0,0,cutscene_click,basic,0,NaN,-420.991405,66.685314,373.0,268.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
3750,20090314363702160,0,0,cutscene_click,basic,0,NaN,-72.991405,20.685314,721.0,314.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
4816,20090314441803444,0,0,cutscene_click,basic,0,NaN,-377.991405,-111.314686,416.0,446.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
5605,20090315081004164,0,0,cutscene_click,basic,0,NaN,-238.991405,-133.314686,555.0,468.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
7132,20090315085850788,0,0,cutscene_click,basic,0,NaN,-412.658072,47.463092,624.0,470.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
8085,20090315101457836,0,0,cutscene_click,basic,0,NaN,-181.257030,-211.017811,713.0,635.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
9185,20090315170769824,0,0,cutscene_click,basic,0,NaN,-374.991405,73.685314,419.0,261.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
10172,20090317080721164,0,0,cutscene_click,basic,0,NaN,-466.991405,36.685314,327.0,298.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4


In [11]:
train[train["level_group"]=="0-4"].groupby(["session_id"]).tail(1).head(10)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
164,20090312431273200,164,194860,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
1019,20090312433251036,138,233752,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
2876,20090314121766812,164,393360,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
3895,20090314363702160,145,217574,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
4922,20090314441803444,106,176953,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
5792,20090315081004164,187,341352,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
7298,20090315085850788,166,425423,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
8248,20090315101457836,163,269374,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
9342,20090315170769824,157,272762,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4
10323,20090317080721164,151,328805,checkpoint,basic,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap1_finale_c,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,0-4


In [12]:
train[train["level_group"]=="5-12"].groupby(["session_id"]).head(1).head(10)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
165,20090312431273200,175,221485,navigate_click,undefined,5,NaN,280.442178,-19.705837,653.0,321.0,NaN,NaN,boss,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
1020,20090312433251036,150,266883,navigate_click,undefined,5,NaN,386.502784,-372.736140,723.0,554.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
2877,20090314121766812,173,437423,navigate_click,undefined,5,NaN,531.957330,-313.645231,819.0,515.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
3896,20090314363702160,156,247100,navigate_click,undefined,5,NaN,300.139148,-274.251291,666.0,489.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
4923,20090314441803444,114,218953,navigate_click,undefined,5,NaN,515.290663,-292.433109,808.0,501.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
5793,20090315081004164,200,376537,navigate_click,undefined,5,NaN,281.957330,-263.645231,654.0,482.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
7299,20090315085850788,174,480433,navigate_click,undefined,5,NaN,393.826017,-237.045904,1191.0,760.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
8249,20090315101457836,169,287483,navigate_click,undefined,5,NaN,416.048239,-306.779700,864.0,594.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
9343,20090315170769824,173,312977,navigate_click,undefined,5,NaN,531.957330,-309.099776,819.0,512.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12
10324,20090317080721164,157,350326,navigate_click,undefined,5,NaN,-58.951761,-196.978564,429.0,438.0,NaN,NaN,NaN,tunic.capitol_0.hall,NaN,NaN,NaN,NaN,5-12


In [13]:
train[train["level_group"]=="5-12"].groupby(["session_id"]).tail(1).head(10)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
460,20090312431273200,470,499235,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
1414,20090312433251036,544,817609,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
3261,20090314121766812,557,1017051,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
4191,20090314363702160,451,552723,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
5180,20090314441803444,371,550531,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
6233,20090315081004164,640,1409898,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
7619,20090315085850788,494,1135963,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
8627,20090315101457836,547,743638,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
9678,20090315170769824,508,775821,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12
11009,20090317080721164,842,1273208,checkpoint,basic,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap2_finale_c,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,5-12


In [15]:
train[train["level_group"]=="13-22"].groupby(["session_id"]).head(1).head(10)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
461,20090312431273200,512,836732,navigate_click,undefined,13,NaN,290.153549,-204.499365,651.0,445.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
1415,20090312433251036,579,1176483,navigate_click,undefined,13,NaN,473.486883,-259.044820,772.0,481.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
3262,20090314121766812,595,1398195,navigate_click,undefined,13,NaN,550.759610,-252.984214,823.0,477.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
4192,20090314363702160,493,713569,navigate_click,undefined,13,NaN,526.517186,-224.196335,807.0,458.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
5181,20090314441803444,409,759652,navigate_click,undefined,13,NaN,358.335368,-231.772093,696.0,463.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
6234,20090315081004164,687,1931307,navigate_click,undefined,13,NaN,496.214155,-384.802396,787.0,564.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
7620,20090315085850788,532,1536971,navigate_click,undefined,13,NaN,532.493617,-165.442126,1327.0,686.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
8628,20090315101457836,596,998455,navigate_click,undefined,13,NaN,198.321163,-249.006941,687.0,552.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
9679,20090315170769824,551,961074,navigate_click,undefined,13,NaN,426.517186,-310.559972,741.0,515.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22
11010,20090317080721164,878,1687952,navigate_click,undefined,13,NaN,587.123246,-266.620578,847.0,486.0,NaN,NaN,NaN,tunic.capitol_1.hall,NaN,NaN,NaN,NaN,13-22


In [16]:
train[train["level_group"]=="13-22"].groupby(["session_id"]).tail(1).head(10)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
880,20090312431273200,931,1272679,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
2711,20090312433251036,1875,3815334,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
3749,20090314121766812,1082,2133782,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
4815,20090314363702160,1116,4208427,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
5604,20090314441803444,832,1275750,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
7131,20090315081004164,1584,2974641,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
8084,20090315085850788,996,2554792,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
9184,20090315101457836,1152,1569636,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
10171,20090315170769824,1043,1578026,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
11525,20090317080721164,1393,2491640,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [30]:
end0_4 = train[train["level_group"]=="0-4"].groupby(["session_id"]).tail(1)
start5_12 = train[train["level_group"]=="5-12"].groupby(["session_id"]).head(1)
df = end0_4.merge(start5_12, on="session_id", how="left")

In [37]:
((df["elapsed_time_y"] - df["elapsed_time_x"]) / 1000).median()

30.378

In [39]:
end5_12 = train[train["level_group"]=="5-12"].groupby(["session_id"]).tail(1)
start13_22 = train[train["level_group"]=="13-22"].groupby(["session_id"]).head(1)
df = end5_12.merge(start13_22, on="session_id", how="left")
((df["elapsed_time_y"] - df["elapsed_time_x"]) / 1000).median()

273.457

In [40]:
train[train["level_group"]=="0-4"].groupby(["session_id"]).head(1)["fqid"].value_counts()

intro        11773
doorblock        1
Name: fqid, dtype: int64

In [41]:
tmp = train[train["level_group"]=="0-4"].groupby(["session_id"]).head(1)
tmp[tmp["fqid"]=="doorblock"]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
5326724,21030209535891704,0,0,navigate_click,undefined,0,NaN,-348.881529,62.731209,456.0,320.0,NaN,NaN,doorblock,tunic.historicalsociety.closet,NaN,NaN,NaN,NaN,0-4


In [42]:
train[train["session_id"]==21030209535891704]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
5326724,21030209535891704,0,0,navigate_click,undefined,0,NaN,-348.881529,62.731209,456.0,320.0,NaN,NaN,doorblock,tunic.historicalsociety.closet,NaN,NaN,NaN,NaN,0-4
5326725,21030209535891704,1,1854,observation_click,basic,0,NaN,-321.724567,-111.870991,587.0,607.0,NaN,I feel like I'm forgetting something.,doorblock,tunic.historicalsociety.closet,tunic.historicalsociety.closet.doorblock,NaN,NaN,NaN,0-4
5326726,21030209535891704,2,3089,navigate_click,undefined,0,NaN,-519.531355,-217.868119,319.0,755.0,NaN,NaN,NaN,tunic.historicalsociety.closet,NaN,NaN,NaN,NaN,0-4
5326727,21030209535891704,3,4057,navigate_click,undefined,0,NaN,-808.632958,-232.862793,34.0,728.0,NaN,NaN,NaN,tunic.historicalsociety.closet,NaN,NaN,NaN,NaN,0-4
5326728,21030209535891704,4,4792,navigate_click,undefined,0,NaN,-899.217261,-246.400580,32.0,728.0,NaN,NaN,notebook,tunic.historicalsociety.closet,NaN,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5328185,21030209535891704,1518,2116349,navigate_click,undefined,22,NaN,866.680186,73.796820,1190.0,384.0,NaN,NaN,tomap,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
5328186,21030209535891704,1519,2118587,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,334.0,NaN,tunic.wildlife,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
5328187,21030209535891704,1520,2119723,map_click,undefined,22,NaN,439.666916,28.388162,592.0,429.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
5328188,21030209535891704,1521,2120791,navigate_click,undefined,22,NaN,272.488073,-21.547946,599.0,428.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [20]:
train[train["level_group"]=="0-4"].groupby(["session_id"]).tail(1)["fqid"].value_counts()

chap1_finale_c        11640
gramps                   20
groupconvo               14
intro                    14
teddy                    10
retirement_letter         6
notebook                  5
chap1_finale              4
tobasement                4
toentry                   3
doorblock                 3
tunic                     3
janitor                   2
tocollection              2
directory                 2
photo                     2
cs                        2
block_tomap2              1
report                    1
block_tomap1              1
tomap                     1
tunic.hub.slip            1
block_tocollection        1
plaque                    1
outtolunch                1
togrampa                  1
Name: fqid, dtype: int64

In [62]:
train[train["level_group"]=="0-4"].groupby(["session_id"]).tail(1)["event_name"].value_counts()

checkpoint            11442
cutscene_click          226
navigate_click           50
person_click             26
object_click             15
notebook_click           10
observation_click         5
object_hover              3
notification_click        2
Name: event_name, dtype: int64

In [43]:
labels["question"] = labels["session_id"].apply(lambda x: x.split("_")[1].replace("q", "")).astype(int)
labels["session_id"] = labels["session_id"].apply(lambda x: x.split("_")[0]).astype(int)

In [45]:
tmp = train[train["level_group"]=="0-4"].groupby(["session_id"]).tail(1)
tail_not_fin_sessions = tmp[tmp["fqid"]!="chap1_finale_c"]["session_id"].tolist()

In [50]:
labels[(labels["session_id"].isin(tail_not_fin_sessions))&(labels["question"]<=4)]["correct"].mean()

0.8039568345323741

In [51]:
labels[(labels["question"]<=4)]["correct"].mean()

0.8585618473554631

In [55]:
%time
a = train["elapsed_time"].values[-1]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.72 µs


In [59]:
df = train[(train["session_id"]==22100221145014656)].copy()

In [60]:
%time
a = df["elapsed_time"].values[-1]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [61]:
df

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
13172664,22100221145014656,0,0,cutscene_click,basic,0,NaN,-606.991405,42.685314,187.0,292.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
13172665,22100221145014656,1,2431,person_click,basic,0,NaN,-424.991405,73.685314,369.0,261.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
13172666,22100221145014656,2,5800,person_click,basic,0,NaN,-123.991405,28.685314,670.0,306.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
13172667,22100221145014656,3,9253,person_click,basic,0,NaN,-278.991405,19.685314,515.0,315.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
13172668,22100221145014656,4,11505,person_click,basic,0,NaN,-288.991405,16.685314,505.0,318.0,NaN,Meetings are BORING!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13174206,22100221145014656,1600,5483231,navigate_click,undefined,22,NaN,343.887291,36.701026,483.0,273.0,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
13174207,22100221145014656,1601,5485166,navigate_click,undefined,22,NaN,332.696070,141.493178,545.0,221.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
13174208,22100221145014656,1602,5485917,navigate_click,undefined,22,NaN,369.912859,140.569205,611.0,217.0,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
13174209,22100221145014656,1603,5486753,navigate_click,undefined,22,NaN,252.299653,123.805889,526.0,232.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [ ]:
%time
a = df[df["fqid"]==]

In [5]:
labels
labels["question"] = labels["session_id"].apply(lambda x: x.split("_")[1].replace("q", "")).astype(int)
labels["session_id"] = labels["session_id"].apply(lambda x: x.split("_")[0]).astype(int)

In [17]:
labels_pivot = pd.pivot_table(labels, index="session_id", columns="question", values="correct")

In [8]:
qs = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

In [30]:
xs = []
ys = []
lifts = []
total = len(labels_pivot)
for x, y  in itertools.combinations(qs, 2):
    support_x = labels_pivot[x].sum() / total
    support_y = labels_pivot[y].sum() / total
    support_xy = len(labels_pivot[(labels_pivot[x]==1)&(labels_pivot[y]==1)]) / total
    lift = support_xy / (support_x * support_y)
    xs.append(x)
    ys.append(y)
    lifts.append(lift)

In [32]:
corr = pd.DataFrame({"x":xs, "y":ys, "lift":lifts})

In [40]:
corr.sort_values("lift", ascending=False).head(50)

,x,y,lift
69,5,13,1.221265
71,5,15,1.216041
121,10,15,1.207374
66,5,10,1.188029
119,10,13,1.166090
111,9,13,1.151427
65,5,9,1.139766
102,8,13,1.139381
113,9,15,1.133832
143,14,15,1.133767


In [12]:
a["level_group"].values[0]

'0-4'

In [13]:
train.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [23]:
labels["session"] = labels["session_id"].apply(lambda x: x.split("_")[0])

In [24]:
labels["session"].value_counts()

20090312431273200    18
21080421144057320    18
21080418362488136    18
21080419064598684    18
21080419184002784    18
                     ..
21020218310048650    18
21020219021855028    18
21020219325225296    18
21020220032589476    18
22100221145014656    18
Name: session, Length: 11779, dtype: int64

In [26]:
session_ids = train["session_id"].unique().tolist()

In [27]:
train[train["session_id"]==session_ids[0]]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,20090312431273200,927,1267357,navigate_click,undefined,22,NaN,927.307255,-10.355929,838.0,335.0,NaN,NaN,tomap,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
877,20090312431273200,928,1268292,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,366.0,NaN,tomap,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
878,20090312431273200,929,1269474,map_click,undefined,22,NaN,457.523005,22.141338,443.0,316.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
879,20090312431273200,930,1270708,navigate_click,undefined,22,NaN,224.190321,-60.268671,404.0,337.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [28]:
train[train["session_id"]==session_ids[1]]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
881,20090312433251036,0,0,cutscene_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
882,20090312433251036,1,218,person_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
883,20090312433251036,2,448,person_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
884,20090312433251036,3,702,person_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
885,20090312433251036,4,948,person_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2707,20090312433251036,1871,3764520,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,49.0,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
2708,20090312433251036,1872,3764686,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,132.0,NaN,tunic.library,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
2709,20090312433251036,1873,3764935,map_click,undefined,22,NaN,476.887585,5.834737,453.0,326.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
2710,20090312433251036,1874,3813349,navigate_click,undefined,22,NaN,239.341836,-17.844428,414.0,309.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [29]:
train[train["session_id"]==session_ids[2]]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
2712,20090314121766812,0,0,cutscene_click,basic,0,NaN,-420.991405,66.685314,373.0,268.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
2713,20090314121766812,1,2317,person_click,basic,0,NaN,-397.991405,71.685314,396.0,263.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2714,20090314121766812,2,4682,person_click,basic,0,NaN,-399.991405,69.685314,394.0,265.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2715,20090314121766812,3,8782,person_click,basic,0,NaN,-400.991405,69.685314,393.0,265.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2716,20090314121766812,4,10749,person_click,basic,0,NaN,-399.991405,67.685314,394.0,267.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,20090314121766812,1078,2130172,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,100.0,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
3746,20090314121766812,1079,2130573,map_click,undefined,22,NaN,453.624385,23.786947,439.0,315.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
3747,20090314121766812,1080,2131805,navigate_click,undefined,22,NaN,283.281230,-29.965640,443.0,317.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
3748,20090314121766812,1081,2133005,navigate_click,undefined,22,NaN,270.625777,0.376447,498.0,306.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [30]:
train[train["session_id"]==session_ids[3]]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
3750,20090314363702160,0,0,cutscene_click,basic,0,NaN,-72.991405,20.685314,721.0,314.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
3751,20090314363702160,1,914,person_click,basic,0,NaN,-209.991405,169.685314,584.0,165.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3752,20090314363702160,2,1929,person_click,basic,0,NaN,-182.991405,169.685314,611.0,165.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3753,20090314363702160,3,2929,person_click,basic,0,NaN,-182.991405,169.685314,611.0,165.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3754,20090314363702160,4,4403,person_click,basic,0,NaN,-182.991405,169.685314,611.0,165.0,NaN,Meetings are BORING!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,20090314363702160,1112,4204831,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,932.0,NaN,tomap,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
4812,20090314363702160,1113,4206393,map_click,undefined,22,NaN,511.913982,-1.469938,479.0,330.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
4813,20090314363702160,1114,4206451,navigate_click,undefined,22,NaN,181.766079,-61.783822,376.0,338.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
4814,20090314363702160,1115,4207180,navigate_click,undefined,22,NaN,262.802419,-17.860023,456.0,316.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [37]:
sorted(train["fqid"].dropna().unique().tolist())

['archivist',
 'archivist_glasses',
 'block',
 'block_0',
 'block_1',
 'block_badge',
 'block_badge_2',
 'block_magnify',
 'block_nelson',
 'block_tocollection',
 'block_tomap1',
 'block_tomap2',
 'boss',
 'businesscards',
 'businesscards.card_0.next',
 'businesscards.card_1.next',
 'businesscards.card_bingo.bingo',
 'businesscards.card_bingo.next',
 'ch3start',
 'chap1_finale',
 'chap1_finale_c',
 'chap2_finale_c',
 'chap4_finale_c',
 'coffee',
 'colorbook',
 'confrontation',
 'crane_ranger',
 'cs',
 'directory',
 'directory.closeup.archivist',
 'door_block_clean',
 'door_block_talk',
 'doorblock',
 'expert',
 'flag_girl',
 'fox',
 'glasses',
 'gramps',
 'groupconvo',
 'groupconvo_flag',
 'intro',
 'janitor',
 'journals',
 'journals.hub.topics',
 'journals.pic_0.next',
 'journals.pic_1.next',
 'journals.pic_2.bingo',
 'journals.pic_2.next',
 'journals_flag',
 'journals_flag.hub.topics',
 'journals_flag.hub.topics_old',
 'journals_flag.pic_0.bingo',
 'journals_flag.pic_0.next',
 'journ

In [17]:
sub.sort_values("session_id")

,session_id,correct,session_level
0,20090109393214576_q1,0,20090109393214576_0-4
27,20090109393214576_q10,0,20090109393214576_5-12
30,20090109393214576_q11,0,20090109393214576_5-12
33,20090109393214576_q12,0,20090109393214576_5-12
36,20090109393214576_q13,0,20090109393214576_5-12
39,20090109393214576_q14,0,20090109393214576_13-22
42,20090109393214576_q15,0,20090109393214576_13-22
45,20090109393214576_q16,0,20090109393214576_13-22
48,20090109393214576_q17,0,20090109393214576_13-22
51,20090109393214576_q18,0,20090109393214576_13-22
